## Explore the dataset

Import Pandas package

In [ ]:
import pandas as pd

In [ ]:
gym = pd.read_csv('data/gym_members_exercise_tracking.csv')

In [ ]:
gym.head(5)

In [ ]:
gym.info()

In [ ]:
gym.shape

In [ ]:
gym.values

In [ ]:
gym.columns

In [ ]:
gym.index

In [ ]:
gym.describe()

## Select a specific column

In [ ]:
gym['Age']

In [ ]:
gym[['Age','Fat_Percentage']]

In [ ]:
gym.loc[:,'Age']

In [ ]:
gym.iloc[:,1]

In [ ]:
gym['Age'].describe()

In [ ]:
gym[['Age']] # ---> dataframe as type

##### -----------------> Mentimiter

### Sort a column

In [ ]:
gym.sort_values(gym.columns[3])

##### -----------------> Mentimiter

In [ ]:
gym.sort_values(['Height (m)','Weight (kg)'], ascending=[False,True])

## Select a row

In [ ]:
gym.iloc[0,:]

In [ ]:
gym[gym['Age'] == 28]

## Subsetting based on multiple conditions

In [ ]:
cond1 = gym['Age'] == 18

In [ ]:
cond2 = gym['Gender'] == 'Male'

In [ ]:
cond3 = gym['Height (m)'] > 1.80

In [ ]:
gym[cond1 & cond2 & cond3]

In [ ]:
ismale = gym['Gender'].isin(['Male'])

In [ ]:
gymmale = gym[ismale]

##### -----------------> Mentimiter

## Adding columns

In [ ]:
gym['Burned_by_duration'] = gym['Calories_Burned'] / gym['Session_Duration (hours)']

In [ ]:
gym

## Column satistics

### Simple statistics

In [ ]:
gym.Age.mean()

In [ ]:
numeric_columns = gym.select_dtypes(include=['number']).columns
gym[numeric_columns].mean()

#### Count occurrencies

In [ ]:
gym['Gender'].value_counts()

In [ ]:
male = gym['Gender'] == 'Male'

In [ ]:
gym[male]['Workout_Type'].value_counts(normalize=True)

### Powerful and flexible aggregation method .agg() 

Togheter with the usual statistic methods .mean() .min() .max()

#### Apply multiple functions at once

In [ ]:
stats = ['mean','sum','min','max']

In [ ]:
gym['Calories_Burned'].agg(stats)

#### Apply multiple functions to multiple columns

In [ ]:
stats = ['mean','sum','min','max']

In [ ]:
columns = ['Calories_Burned', 'Fat_Percentage']

In [ ]:
gym[columns].agg(stats)

#### Custom functions

In [ ]:
rangefun = lambda x: x.max() - x.min()

In [ ]:
gym['Calories_Burned'].agg(rangefun)

#### Different functions to different columns

In [ ]:
functions = {'Calories_Burned':'mean', 'Fat_Percentage':'max'}

In [ ]:
gym.agg(functions)

In [ ]:
gym.agg(Mean_Calories = ('Calories_Burned', 'mean'), Max_Fat = ('Fat_Percentage', 'max'))

In [ ]:
functions = {'Mean_calories':('Calories_Burned','mean'),
             'Max_Flat':('Fat_Percentage','max')}

In [ ]:
gym.agg(**functions) 

The ** operator "unpacks" the 'functions' dictionary, this means that the keys and values of the dictionary are passed as keyword arguments to the .agg() function.

## GroupBy

In [ ]:
numeric_columns = gym.select_dtypes(include=['number']).columns
gym.groupby(['Workout_Type'])[numeric_columns].mean()

In [ ]:
gymc = gym.iloc[:,[0,4,5,6,12]]

In [ ]:
gymc = gymc.copy()

In [ ]:
gymc.head(1)

In [ ]:
# Define the limits
bins = [49, 56, 62, 68, 74]
bins1 = [17, 29, 39, 49, 59]

# Define labels
labels = ['50-56','57-62','63-68','69-74']
labels1 = ['18-29', '30-39', '40-49', '50-59']


# Use the function cut to create the new columns
gymc.loc[:,'BeatsRest_Class'] = pd.cut(gymc.loc[:,'Resting_BPM'], bins=bins, labels=labels, right=True) 
# right=True means closed to the right open to the left
gymc.loc[:,'Age_Class'] = pd.cut(gymc.loc[:,'Age'], bins=bins1, labels=labels1, right=True)

In [ ]:
gymc.head(3)

In [ ]:
numeric_columns = gymc.select_dtypes(include=['number']).columns
gymc.groupby(['Age_Class', 'BeatsRest_Class'],observed=True)[numeric_columns].mean()

##### -----------------> Mentimiter

## Pivot tables

In [ ]:
gym.pivot_table(values='Age', index='Workout_Type')

In [ ]:
rangefun = lambda x: str(x.min()) + ' - ' + str(x.max())
rangefun.__name__ = 'Range'

In [ ]:
meanmed = lambda x: str(x.mean()) + ' - ' + str(x.median())
meanmed.__name__ = 'Mean - Median'

In [ ]:
gym.pivot_table(values='Calories_Burned', index='Workout_Type', aggfunc=[rangefun,meanmed])

Using Groupby

In [ ]:
gym.groupby(['Workout_Type'])['Calories_Burned'].agg(['min','max'])

In [ ]:
gym.pivot_table(values='Calories_Burned', index='Workout_Type',columns='Experience_Level')

In [ ]:
gym.pivot_table(values='Calories_Burned', index='Workout_Type',columns='Experience_Level', margins=True)

In [ ]:
gym.pivot_table(values='Burned_by_duration', index='Workout_Type',columns='Experience_Level')

## Working with indexes

In [ ]:
gym.set_index('Workout_Type')

Simpler to subset

In [ ]:
gym.set_index('Workout_Type').loc['Yoga']

In [ ]:
gym.set_index(['Workout_Type','Age']).loc[['Yoga','HIIT']].sort_index(level=["Workout_Type", "Age"], ascending=[True, False])

In [ ]:
gym.columns

In [ ]:
gym.set_index(['Workout_Type','Age']).loc[[('Yoga',20),('HIIT',30)],'Weight (kg)':'Resting_BPM'] # --> NOTE:Resting_BPM included

In [ ]:
gym.pivot_table('Fat_Percentage',index='Workout_Type',columns='Experience_Level')

In [ ]:
gym.pivot_table('Fat_Percentage',index='Workout_Type',columns='Experience_Level').loc['Cardio':'Strength']

In [ ]:
gym.pivot_table('Fat_Percentage',index='Workout_Type',columns='Experience_Level').mean(axis='index')

In [ ]:
gym.pivot_table('Fat_Percentage',index='Workout_Type',columns='Experience_Level').mean(axis='columns')

## Working with missing values

In [ ]:
gymfiltered = gym.filter(like='BPM').iloc[1:20].copy()

In [ ]:
import random
import numpy as np

# Numero di NaN da inserire
n_nan = 10

# Ottieni gli indici delle righe e delle colonne
rows = gymfiltered.index
cols = gymfiltered.columns

# Genera una lista di tuple (riga, colonna) casuali
random_indices = random.sample([(r, c) for r in rows for c in cols], n_nan)

# Inserisci NaN nelle posizioni casuali
for r, c in random_indices:
    gymfiltered.loc[r, c] = np.nan

gymfiltered

In [ ]:
gymfiltered.isna().sum()

In [ ]:
gymfiltered.dropna().isna().sum()

In [ ]:
gymfiltered.fillna(0)

<span style="background-color:yellow;">Be carefull!</span>

In [ ]:
gymfiltered.isna().sum()

In [ ]:
gymfiltered.dropna(inplace=True)

In [ ]:
gymfiltered.isna().sum()

In [ ]:
gymfiltered

#### Saving to csv

In [ ]:
gymfiltered.to_csv('gymfiltered.csv')

## Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
gym['Age'].hist(bins=15)

In [ ]:
gym.columns

In [ ]:
gym.groupby('Workout_Type')['BMI'].mean().plot(kind='bar')

In [ ]:
gym[gym["Gender"]=="Female"]["Weight (kg)"].hist(alpha=0.7,color='red')
gym[gym["Gender"]=="Male"]["Weight (kg)"].hist(alpha=0.7,color='blue')
plt.legend(["Female", "Male"])

In [ ]:
# Calcolo delle frequenze di 'Workout_Type' raggruppate per 'Gender'
grouped_data = gym.groupby(['Workout_Type', 'Gender']).size().unstack()

# Imposta la figura
plt.figure(figsize=(12, 6))

# Creazione del grafico a barre
x_labels = grouped_data.index
x = range(len(x_labels))
bar_width = 0.35

# Disegna le barre per ciascun genere
for i, gender in enumerate(grouped_data.columns):
    plt.bar(
        [pos + i * bar_width for pos in x],
        grouped_data[gender],
        width=bar_width,
        label=gender
    )

# Etichette e titolo
plt.xticks([pos + bar_width / 2 for pos in x], x_labels)
plt.title('Workout Type by Gender')
plt.xlabel('Workout Type')
plt.ylabel('Count')
plt.legend(title='Gender')

# Mostra il grafico
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calcolo delle frequenze di 'Workout_Type' raggruppate per 'Gender'
grouped_data = gym.groupby(['Workout_Type', 'Gender']).size().unstack()

# Imposta la figura
plt.figure(figsize=(12, 6))

# Creazione del grafico a barre
grouped_data.plot(kind='bar', width=0.8, ax=plt.gca())  # Usa direttamente Pandas per il grafico a barre

# Etichette e titolo
plt.title('Workout Type by Gender')
plt.xlabel('Workout Type')
plt.ylabel('Count')
plt.legend(title='Gender')

# Mostra il grafico
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dati da plottare
x = gym['Session_Duration (hours)']
y = gym['Calories_Burned']

# Calcolo della linea di regressione
coefficients = np.polyfit(x, y, deg=1)  # Regressione lineare
regression_line = np.poly1d(coefficients)

# Imposta la figura
plt.figure(figsize=(12, 6))

# Scatter plot (punti dati)
plt.scatter(x, y, color='blue', alpha=0.7, label='Data points')

# Linea di regressione
x_vals = np.linspace(x.min(), x.max(), 100)
plt.plot(x_vals, regression_line(x_vals), color='red', label='Regression line')

# Titolo e etichette
plt.title('Calories Burned by Session Duration')
plt.xlabel('Session Duration (hours)')
plt.ylabel('Calories Burned')
plt.legend()

# Mostra il grafico
plt.tight_layout()
plt.show()


##### -----------------> Mentimiter